In [1]:
import os
import functools
from warnings import simplefilter

In [2]:
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

#### Configure path parameters and read data files

The `RANDOM_SEED`, `NUM_EPOCHS`, `TRAIN_FRAC`, and `VAL_FRAC` are the same for all models during evaluaiton

In [3]:
MIDS_USER = os.getenv("MIDS_USER", None)  # this can be replaced, with e.g. `m25...`

if MIDS_USER is None:
  ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), *['..'] * 1)) 
else:
  ROOT_DIR = f"/home/mids/{MIDS_USER}/Capstone"

IMAGES_DIR = os.path.join(ROOT_DIR, "data", "datasets", "aircraft", "images")
if not os.path.exists(IMAGES_DIR):
  ROOT_DIR = os.path.join(ROOT_DIR, "lc-14")
  IMAGES_DIR = os.path.join(ROOT_DIR, "data", "datasets", "aircraft", "images")

RADOM_SEED = 2020
TRAIN_FRAC = 0.8
VAL_FRAC = 0.1

NUM_EPOCHS = 10   # This can be adjusted as needed 

In [4]:
image_dir = IMAGES_DIR
labels_fp = os.path.join(ROOT_DIR, "data", "datasets", "aircraft", "annotations.csv")

#### Load utility functions

In [5]:
os.chdir(ROOT_DIR)

In [6]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from src.utilities.data.aircraft_dataloader import get_dataloader

#### Load models and transformations

In [7]:
from src.models.baseline.cnn import BaselineCNN
from src.models.alpha.model import AlphaModel
# from src.models.beta.model import BetaModel
# from src.models.final.model import FinalModel

from src.transformations.baseline.transformations import baseline_transformations
from src.transformations.alpha.transformations import alpha_transformations
# from src.transformations.beta.transformations import beta_transformations
# from src.transformations.final.transformations import final_transformations

#### Define evaluation configuration

In [8]:
# We will use `functools.partial` to create a partial function that we can use to create instances of our models
# These will be instantiated in the training loop
default_criterion = functools.partial(nn.MSELoss)

baseline_model = functools.partial(BaselineCNN)
baseline_model_optimizer = functools.partial(
  torch.optim.Adam, 
  lr=0.01,
)
alpha_model = functools.partial(AlphaModel)
alpha_model_optimizer = functools.partial(
  torch.optim.Adam, 
  lr=1e-5,
)
alpha_model_criterion = functools.partial(nn.SmoothL1Loss, beta=1.0)
# beta_model = functools.partial(BetaModel)
# beta_model_optimizer = functools.partial(
#   torch.optim.Adam, 
#   lr=0.01,
# )
# final_model = functools.partial(FinalModel)
# final_model_optimizer = functools.partial(
#   torch.optim.Adam, 
#   lr=0.01,
# )

In [9]:
model_configurations = {
  "Baseline CNN": {
    "model_callable": baseline_model,
    "optimizer_callable": baseline_model_optimizer,
    "criterion_callable": default_criterion,
    "transformations": baseline_transformations,
    "dataloader_kwargs": {
      "batch_size": 4,
      "shuffle": True,
    },
  },
  "Alpha Model": {
    "model_callable": alpha_model,
    "optimizer_callable": alpha_model_optimizer,
    "criterion_callable": alpha_model_criterion,
    "transformations": alpha_transformations,
    "dataloader_kwargs": {
      "batch_size": 4,
      "shuffle": True,
    },
  },
  # "Beta Model": {
  #   "model_callable": beta_model,
  #   "optimizer_callable": beta_model_optimizer,
  #   "criterion_callable": default_criterion,
  #   "transformations": beta_transformations,
  #   "dataloader_kwargs": {
  #     "batch_size": 4,
  #     "shuffle": True,
  #   },
  # },
  # "Final Model": {
  #   "model_callable": final_model,
  #   "optimizer_callable": final_model_optimizer,
  #   "criterion_callable": default_criterion,
  #   "transformations": final_transformations,
  #   "dataloader_kwargs": {
  #     "batch_size": 4,
  #     "shuffle": True,
  #   },
  # },
}

### Train the models

In [10]:
for model_name, model_config in model_configurations.items():

  model_config["model"] = model_config["model_callable"]()
  model_config["optimizer"] = model_config["optimizer_callable"](params=model_config["model"].parameters())
  model_config["criterion"] = model_config["criterion_callable"]()
  model_config["model"] .train()

  # Training loop
  for epoch in range(NUM_EPOCHS):

    dataloader = get_dataloader(
      image_dir=image_dir,
      labels_fp=labels_fp,
      transformations=model_config["transformations"],
      mode='train',
      train_frac=TRAIN_FRAC,
      val_frac=VAL_FRAC,
      seed=RADOM_SEED + epoch,
      batch_size=model_config["dataloader_kwargs"]["batch_size"],
      shuffle=model_config["dataloader_kwargs"]["shuffle"],
      num_workers=1,
    )

    for i, (images, targets) in enumerate(dataloader):

      model_config["optimizer"].zero_grad()
      outputs = model_config["model"](images)
      loss = model_config["criterion"](outputs.squeeze(), targets)  # Ensure outputs are squeezed to match counts shape
      loss.backward()
      model_config["optimizer"].step()

    print(f"Model {model_name} epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}")


Model Baseline CNN epoch [1/10], Loss: 954.1287
Model Baseline CNN epoch [2/10], Loss: 547.9930
Model Baseline CNN epoch [3/10], Loss: 2333.2686
Model Baseline CNN epoch [4/10], Loss: 381.1278
Model Baseline CNN epoch [5/10], Loss: 189.6175
Model Baseline CNN epoch [6/10], Loss: 395.2578
Model Baseline CNN epoch [7/10], Loss: 95.9918
Model Baseline CNN epoch [8/10], Loss: 177.0329
Model Baseline CNN epoch [9/10], Loss: 0.9389
Model Baseline CNN epoch [10/10], Loss: 39.0845
Model Alpha Model epoch [1/10], Loss: 27.1912
Model Alpha Model epoch [2/10], Loss: 12.7628
Model Alpha Model epoch [3/10], Loss: 0.4316
Model Alpha Model epoch [4/10], Loss: 2.6640
Model Alpha Model epoch [5/10], Loss: 15.6236
Model Alpha Model epoch [6/10], Loss: 2.4050
Model Alpha Model epoch [7/10], Loss: 0.7829
Model Alpha Model epoch [8/10], Loss: 11.8301
Model Alpha Model epoch [9/10], Loss: 2.2555
Model Alpha Model epoch [10/10], Loss: 5.7584


### Evaluate the models

In [11]:
model_losses = {
  model_name: [] for model_name in model_configurations.keys()
}
model_predictions = {
  model_name: [] for model_name in model_configurations.keys()
}


In [12]:
for model_name, model_config in model_configurations.items():
  
  val_dataloader = get_dataloader(
    image_dir,
    labels_fp,
    transformations=model_config["transformations"],
    mode='val',
    train_frac=TRAIN_FRAC,
    val_frac=VAL_FRAC,
    seed=RADOM_SEED,
    batch_size=1,
    shuffle=True,
    num_workers=1,
  )
  true_labels = []

  model_config["model"].eval()

  for i, (val_images, targets) in enumerate(val_dataloader):
    predicted_counts = model_config["model"](val_images)
    # We validate based on the mean absolute error
    model_losses[model_name].append(torch.abs(predicted_counts - targets).item())
    model_predictions[model_name].append(predicted_counts.item())
    true_labels.append(targets.item())
    print(f"{model_name} - val image {i+1}, predicted count: {predicted_counts.item():.4f}, true count: {targets.item():.4f}")


Baseline CNN - val image 1, predicted count: 29.8421, true count: 49.0000
Baseline CNN - val image 2, predicted count: 29.8421, true count: 19.0000
Baseline CNN - val image 3, predicted count: 29.8421, true count: 52.0000
Baseline CNN - val image 4, predicted count: 29.8421, true count: 27.0000
Baseline CNN - val image 5, predicted count: 29.8421, true count: 26.0000
Baseline CNN - val image 6, predicted count: 29.8421, true count: 24.0000
Baseline CNN - val image 7, predicted count: 29.8421, true count: 15.0000
Baseline CNN - val image 8, predicted count: 29.8421, true count: 39.0000
Baseline CNN - val image 9, predicted count: 29.8421, true count: 31.0000
Baseline CNN - val image 10, predicted count: 29.8421, true count: 30.0000
Alpha Model - val image 1, predicted count: 26.4065, true count: 15.0000
Alpha Model - val image 2, predicted count: 28.8873, true count: 26.0000
Alpha Model - val image 3, predicted count: 24.7642, true count: 19.0000
Alpha Model - val image 4, predicted cou

### Compare the models

In [13]:
for model_name, model_losses in model_losses.items():
  print(f"Model {model_name} MAE: {sum(model_losses) / len(model_losses):.4f}")


Model Baseline CNN MAE: 9.0000
Model Alpha Model MAE: 9.0154


In [14]:
model_predictions

{'Baseline CNN': [29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578,
  29.842121124267578],
 'Alpha Model': [26.40654945373535,
  28.887264251708984,
  24.764240264892578,
  27.365055084228516,
  25.633752822875977,
  30.141298294067383,
  26.02460479736328,
  26.03555679321289,
  30.98990249633789,
  25.263553619384766]}

In [15]:
plt.figure(figsize=(8, 8))
for model_name, model_prediction in model_predictions.items():
  plt.scatter(model_prediction, true_labels, label=f"{model_name} predictions")

# Set the x and y axis labels and title
plt.xlim(0, 50)
plt.ylim(0, 50)
plt.axline((0, 0), slope=1, color='red', linestyle='--', label='y=x')
plt.grid()
plt.xlabel("Predicted Counts")
plt.ylabel("True Counts")
plt.title("Model Predictions vs True Counts")
plt.legend()
plt.show()


### Use the models operationally against new data

This section uses new data (of a similar form, aerial images of aircraft and airports) to assess the model performance in a real-world scenario.

In [16]:
boneyard_aircraft_dir = "/home/cjellen/projects/github/lc-14/data/processed/boneyard/boneyard-aircraft"
boneyard_no_aircraft_dir = "/home/cjellen/projects/github/lc-14/data/processed/boneyard/boneyard-no-aircraft"

In [17]:
model_boneyard_predictions = {
  model_name: [] for model_name in model_configurations.keys()
}
model_boneyard_no_aircraft_predictions = {
  model_name: [] for model_name in model_configurations.keys()
}

In [18]:
for model_name, model_config in model_configurations.items():
  
  # Boneyard with aircraft
  aircraft_dataloader = get_dataloader(
    boneyard_aircraft_dir,
    None,  # No labels for this dataset
    transformations=model_config["transformations"],
    mode='val',
    train_frac=0,
    val_frac=1,
    seed=RADOM_SEED,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    extension=".png",
  )

  model_config["model"].eval()

  for i, (val_images, targets) in enumerate(aircraft_dataloader):
    predicted_counts = model_config["model"](val_images)
    
    model_boneyard_predictions[model_name].append(predicted_counts.item())
    print(f"{model_name} - val image which includes aircraft {i+1}, predicted count: {predicted_counts.item():.4f}")


  # Boneyard without aircraft
  no_aircraft_dataloader = get_dataloader(
    boneyard_no_aircraft_dir,
    None,  # No labels for this dataset
    transformations=model_config["transformations"],
    mode='val',
    train_frac=0,
    val_frac=1,
    seed=RADOM_SEED,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    extension=".png",
  )

  model_config["model"].eval()

  for i, (val_images, targets) in enumerate(no_aircraft_dataloader):
    predicted_counts = model_config["model"](val_images)
    
    model_boneyard_no_aircraft_predictions[model_name].append(predicted_counts.item())
    print(f"{model_name} - val image which does not include aircraft {i+1}, predicted count: {predicted_counts.item():.4f}")


Baseline CNN - val image which includes aircraft 1, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 2, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 3, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 4, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 5, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 6, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 7, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 8, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 9, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 10, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 11, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 12, predicted count: 29.8421
Baseline CNN - val image which includes aircraft 13, predicted count: 29.